### Organizing the dataset 

In [19]:
import torch
import torchvision
from torchvision.models.detection import fasterrcnn_resnet50_fpn
from torchvision.transforms import functional as F
from PIL import Image
import matplotlib.pyplot as plt
import numpy as np
import os
import shutil
from sklearn.model_selection import train_test_split
import random

In [5]:


# Define the paths
base_dir = 'recycling/images\images'  # Adjust base_dir to match your current directory structure
target_base_dir = 'dataset'

# Define the classes and subclasses
classes = {
    'plastic': ['plastic_water_bottles', 'plastic_soda_bottles', 'plastic_detergent_bottles', 'plastic_shopping_bags',
                'plastic_trash_bags', 'plastic_food_containers', 'disposable_plastic_cutlery', 'plastic_straws',
                'plastic_cup_lids'],
    'paper and carboard': ['newspaper', 'office_paper', 'magazines', 'cardboard_boxes', 'cardboard_packaging',
                           'paper_cups'],
    'glass': ['glass_beverage_bottles', 'glass_cosmetic_containers', 'glass_food_jars'],
    'metal': ['aerosol_cans', 'aluminum_food_cans', 'aluminum_soda_cans', 'steel_food_cans'],
    'textiles': ['shoes', 'clothing'],
    'organic': ['eggshells', 'food_waste', 'tea_bags', 'coffee_grounds'],
    'general trash': ['styrofoam_cups', 'styrofoam_food_containers']
}

# Create target directories if they don't exist
for class_name, subclasses in classes.items():
    for subclass in subclasses:
        os.makedirs(os.path.join(target_base_dir, 'train', class_name, subclass), exist_ok=True)
        os.makedirs(os.path.join(target_base_dir, 'test', class_name, subclass), exist_ok=True)

# Function to move files to the target directory
def move_files(file_list, target_dir):
    for file in file_list:
        # Construct the target file path
        target_file = os.path.join(target_dir, os.path.basename(file))
        
        # Check if the file already exists in the target directory
        if os.path.exists(target_file):
            # Handle existing files (e.g., overwrite or skip)
            print(f"File '{target_file}' already exists. Skipping.")
            continue
        
        # Move the file to the target directory
        shutil.move(file, target_dir)
        print(f"Moved {file} to {target_dir}")

# Organize the files
for class_name, subclasses in classes.items():
    for subclass in subclasses:
        # Process 'default' and 'real_world' folders separately
        for condition in ['default', 'real_world']:
            # Get all PNG files in the current subclass and condition directory
            subclass_dir = os.path.join(base_dir, subclass, condition)
            if os.path.exists(subclass_dir):
                files = [os.path.join(subclass_dir, f) for f in os.listdir(subclass_dir) if
                         os.path.isfile(os.path.join(subclass_dir, f)) and f.endswith('.png')]
                
                # Ensure there are files to process
                if len(files) == 0:
                    print(f"No .png files found in {subclass_dir}. Skipping.")
                    continue
                
                # Determine the target directory (train or test)
                target_directory = os.path.join(target_base_dir, 'train' if condition == 'default' else 'test',
                                                class_name, subclass)
                
                # Move files to the respective directory without splitting
                move_files(files, target_directory)

print("Dataset organized successfully.")


No .png files found in recycling/images\images\plastic_water_bottles\default. Skipping.
No .png files found in recycling/images\images\plastic_water_bottles\real_world. Skipping.
No .png files found in recycling/images\images\plastic_soda_bottles\default. Skipping.
No .png files found in recycling/images\images\plastic_soda_bottles\real_world. Skipping.
No .png files found in recycling/images\images\plastic_detergent_bottles\default. Skipping.
No .png files found in recycling/images\images\plastic_detergent_bottles\real_world. Skipping.
No .png files found in recycling/images\images\plastic_shopping_bags\default. Skipping.
No .png files found in recycling/images\images\plastic_shopping_bags\real_world. Skipping.
No .png files found in recycling/images\images\plastic_trash_bags\default. Skipping.
No .png files found in recycling/images\images\plastic_trash_bags\real_world. Skipping.
No .png files found in recycling/images\images\plastic_food_containers\default. Skipping.
No .png files f

### Object detection training 

In [14]:
# Load a pre-trained Faster R-CNN model
model = fasterrcnn_resnet50_fpn(pretrained=True)
model.eval()


FasterRCNN(
  (transform): GeneralizedRCNNTransform(
      Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
      Resize(min_size=(800,), max_size=1333, mode='bilinear')
  )
  (backbone): BackboneWithFPN(
    (body): IntermediateLayerGetter(
      (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
      (bn1): FrozenBatchNorm2d(64, eps=0.0)
      (relu): ReLU(inplace=True)
      (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
      (layer1): Sequential(
        (0): Bottleneck(
          (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn1): FrozenBatchNorm2d(64, eps=0.0)
          (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
          (bn2): FrozenBatchNorm2d(64, eps=0.0)
          (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn3): FrozenBatchNorm2d(256, eps=0.0)
          (relu): ReLU(

In [15]:
# Define the classes and subclasses
classes = {
    'plastic': ['plastic_water_bottles', 'plastic_soda_bottles', 'plastic_detergent_bottles', 'plastic_shopping_bags',
                'plastic_trash_bags', 'plastic_food_containers', 'disposable_plastic_cutlery', 'plastic_straws',
                'plastic_cup_lids'],
    'paper and carboard': ['newspaper', 'office_paper', 'magazines', 'cardboard_boxes', 'cardboard_packaging',
                           'paper_cups'],
    'glass': ['glass_beverage_bottles', 'glass_cosmetic_containers', 'glass_food_jars'],
    'metal': ['aerosol_cans', 'aluminum_food_cans', 'aluminum_soda_cans', 'steel_food_cans'],
    'textiles': ['shoes', 'clothing'],
    'organic': ['eggshells', 'food_waste', 'tea_bags', 'coffee_grounds'],
    'general trash': ['styrofoam_cups', 'styrofoam_food_containers']
}

In [6]:
def run_faster_rcnn(image_path, model):
    # Load an image
    image = Image.open(image_path).convert("RGB")

    # Transform the image to a tensor
    image_tensor = F.to_tensor(image)

    # Add a batch dimension
    image_tensor = image_tensor.unsqueeze(0)

    # Perform object detection
    with torch.no_grad():
        predictions = model(image_tensor)

    # Get the predicted bounding boxes, labels, and scores
    boxes = predictions[0]['boxes'].cpu().numpy()
    labels = predictions[0]['labels'].cpu().numpy()
    scores = predictions[0]['scores'].cpu().numpy()

    return image, boxes, labels, scores

def visualize_detections(image, boxes, labels, scores, threshold=0.5):
    plt.figure(figsize=(12, 8))
    plt.imshow(image)
    
    # Draw bounding boxes
    for box, label, score in zip(boxes, labels, scores):
        if score > threshold:  # Only show boxes with a confidence score above the threshold
            xmin, ymin, xmax, ymax = box
            plt.gca().add_patch(plt.Rectangle((xmin, ymin), xmax-xmin, ymax-ymin, 
                                              edgecolor='red', facecolor='none', linewidth=2))
            plt.text(xmin, ymin, f'{label}: {score:.2f}', fontsize=12, color='red')
    
    plt.axis('off')
    plt.show()


In [18]:
# Define the paths based on your previous organization
train_image_dir = 'c:\\Users\\anama\\OneDrive\\Documents\\GitHub\\project-cv\\dataset\\train'
test_image_dir = 'c:\\Users\\anama\\OneDrive\\Documents\\GitHub\\project-cv\\dataset\\test'

# Function to get image pathse
def get_image_paths(base_dir,classes):
    image_paths = []
    for class_name, subclasses in classes.items():
        for subclass in subclasses:
            subclass_dir = os.path.join(base_dir, class_name, subclass)
            if os.path.exists(subclass_dir):
                images = [os.path.join(subclass_dir, img) for img in os.listdir(subclass_dir) if img.endswith('.jpg') or img.endswith('.png')]
                image_paths.extend(images)
    return image_paths

train_image_paths = get_image_paths(train_image_dir,classes)
test_image_paths = get_image_paths(test_image_dir, classes)


In [12]:
# Initialize a dictionary to store scores for each class
class_scores = {class_name: [] for class_name in classes.keys()}

# Define a threshold value
threshold = 0.5

# Run the model on each image and store the scores
for image_path in train_image_paths + test_image_paths:
    print("Processing image:", image_path)
    image, boxes, labels, scores = run_faster_rcnn(image_path, model)
    
    # Determine the class based on the image path
    class_name = os.path.basename(os.path.dirname(os.path.dirname(image_path)))
    
    # Filter scores based on the threshold and store them
    filtered_scores = [score for score in scores if score >= threshold]
    class_scores[class_name].extend(filtered_scores)

# Calculate the mean score for each class
mean_scores = {class_name: np.mean(scores) if scores else 0 for class_name, scores in class_scores.items()}

# Print the mean scores
for class_name, mean_score in mean_scores.items():
    print(f"Mean score for {class_name}: {mean_score:.4f}")

Processing image: c:\Users\anama\OneDrive\Documents\GitHub\project-cv\dataset\train\plastic\plastic_water_bottles\Image_1.png
Processing image: c:\Users\anama\OneDrive\Documents\GitHub\project-cv\dataset\train\plastic\plastic_water_bottles\Image_10.png
Processing image: c:\Users\anama\OneDrive\Documents\GitHub\project-cv\dataset\train\plastic\plastic_water_bottles\Image_100.png
Processing image: c:\Users\anama\OneDrive\Documents\GitHub\project-cv\dataset\train\plastic\plastic_water_bottles\Image_101.png
Processing image: c:\Users\anama\OneDrive\Documents\GitHub\project-cv\dataset\train\plastic\plastic_water_bottles\Image_102.png
Processing image: c:\Users\anama\OneDrive\Documents\GitHub\project-cv\dataset\train\plastic\plastic_water_bottles\Image_103.png
Processing image: c:\Users\anama\OneDrive\Documents\GitHub\project-cv\dataset\train\plastic\plastic_water_bottles\Image_104.png
Processing image: c:\Users\anama\OneDrive\Documents\GitHub\project-cv\dataset\train\plastic\plastic_water_b

KeyboardInterrupt: 

In [20]:
train_image_dir = 'c:\\Users\\anama\\OneDrive\\Documents\\GitHub\\project-cv\\dataset\\train'

# Get all image paths
all_image_paths = get_image_paths(train_image_dir, classes)

# Shuffle the dataset
random.shuffle(all_image_paths)

# Calculate split indices
split_index = int(len(all_image_paths) * 0.8)

# Split into training and validation sets
train_image_paths = all_image_paths[:split_index]
val_image_paths = all_image_paths[split_index:]

In [21]:
# Initialize a dictionary to store scores for each class
class_scores = {class_name: [] for class_name in classes.keys()}

# Define a threshold value
threshold = 0.5

# Run the model on each image in the training set and store the scores
for image_path in train_image_paths:
    print("Processing training image:", image_path)
    image, boxes, labels, scores = run_faster_rcnn(image_path, model)
    
    # Determine the class based on the image path
    class_name = os.path.basename(os.path.dirname(os.path.dirname(image_path)))
    
    # Filter scores based on the threshold and store them
    filtered_scores = [score for score in scores if score >= threshold]
    class_scores[class_name].extend(filtered_scores)

# Calculate the mean score for each class in the training set
mean_scores = {class_name: np.mean(scores) if scores else 0 for class_name, scores in class_scores.items()}

# Print the mean scores for the training set
print("Training Set Mean Scores:")
for class_name, mean_score in mean_scores.items():
    print(f"Mean score for {class_name}: {mean_score:.4f}")


Processing training image: c:\Users\anama\OneDrive\Documents\GitHub\project-cv\dataset\train\textiles\shoes\Image_77.png
Processing training image: c:\Users\anama\OneDrive\Documents\GitHub\project-cv\dataset\train\textiles\shoes\Image_205.png
Processing training image: c:\Users\anama\OneDrive\Documents\GitHub\project-cv\dataset\train\paper and carboard\cardboard_packaging\Image_114.png
Processing training image: c:\Users\anama\OneDrive\Documents\GitHub\project-cv\dataset\train\plastic\plastic_water_bottles\Image_188.png
Processing training image: c:\Users\anama\OneDrive\Documents\GitHub\project-cv\dataset\train\glass\glass_cosmetic_containers\Image_239.png
Processing training image: c:\Users\anama\OneDrive\Documents\GitHub\project-cv\dataset\train\glass\glass_beverage_bottles\Image_91.png
Processing training image: c:\Users\anama\OneDrive\Documents\GitHub\project-cv\dataset\train\general trash\styrofoam_cups\Image_123.png
Processing training image: c:\Users\anama\OneDrive\Documents\Git